In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from sklearn.preprocessing import StandardScaler
from scipy.io.arff import loadarff 

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.utils import to_categorical

In [ ]:
raw_data = loadarff('Dry_Bean_Dataset.arff')
df = pd.DataFrame(raw_data[0])

df.Class = pd.factorize(df.Class)[0]
df = df.sample(frac=1).reset_index(drop=True)
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,74923.0,1101.567,385.532964,248.443048,1.551796,0.764676,76197.0,308.860691,0.774254,0.983280,0.775895,0.801127,0.005146,0.001307,0.641804,0.995949,1
1,33752.0,685.368,258.843071,166.339893,1.556109,0.766178,34150.0,207.302632,0.759137,0.988346,0.902946,0.800882,0.007669,0.001946,0.641411,0.998106,6
2,36824.0,689.547,236.361416,198.577267,1.190274,0.542366,37125.0,216.531229,0.767199,0.991892,0.973225,0.916102,0.006419,0.002789,0.839243,0.998930,0
3,52808.0,928.664,332.582478,202.732146,1.640502,0.792733,54100.0,259.301434,0.702683,0.976118,0.769471,0.779661,0.006298,0.001435,0.607871,0.997213,1
4,47124.0,810.080,301.736291,199.665053,1.511212,0.749751,47631.0,244.949261,0.766506,0.989356,0.902394,0.811799,0.006403,0.001715,0.659018,0.995916,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,40166.0,731.962,252.627058,202.940053,1.244836,0.595549,40541.0,226.143626,0.753866,0.990750,0.942088,0.895168,0.006290,0.002491,0.801326,0.997519,0
13607,35315.0,682.216,240.633701,187.269379,1.284960,0.627974,35619.0,212.048236,0.777692,0.991465,0.953510,0.881208,0.006814,0.002534,0.776527,0.997806,6
13608,43460.0,773.873,276.821080,201.064006,1.376781,0.687344,44108.0,235.233906,0.788090,0.985309,0.911927,0.849769,0.006370,0.002049,0.722107,0.994183,5
13609,207561.0,1752.359,667.531421,400.332038,1.667444,0.800210,210438.0,514.076719,0.720660,0.986329,0.849395,0.770116,0.003216,0.000698,0.593079,0.988926,2


In [ ]:
y = df['Class']
x = df.drop('Class',axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state = 1)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

y_train = to_categorical(y_train, 7)
y_test = to_categorical(y_test, 7)

In [ ]:
from itertools import product

activation = ["relu", LeakyReLU()]
loss = ["categorical_crossentropy", "kullback_leibler_divergence"]
optimizer = ["SGD", "adam"]

res = pd.DataFrame(columns=[
    'activation', 'loss', 'optimizer',
    'Точность на обучающем','Точность на тестовом' 
])

for a, l, o in product(activation, loss, optimizer):  
  model = Sequential()
  model.add(Dense(units=28, input_dim = x_train.shape[1], activation = a))
  model.add(Dense(7, activation = "softmax")) 

  model.compile(loss=l, optimizer=o,metrics = ["accuracy"])
  model.fit(x_train, y_train, validation_split=0.2, epochs=10, batch_size=50, verbose=1)
  res.loc[res.shape[0]] = [a,l,o,model.evaluate(x_train, y_train)[1],model.evaluate(x_test, y_test)[1]]

res = res.sort_values('Точность на тестовом', ascending=False)
res.head()

Epoch 1/10
146/146 [==============================] - 1s 4ms/step - loss: 1.4809 - accuracy: 0.5243 - val_loss: 1.0320 - val_accuracy: 0.7029
Epoch 2/10
146/146 [==============================] - 0s 3ms/step - loss: 0.8728 - accuracy: 0.7663 - val_loss: 0.7482 - val_accuracy: 0.8147
Epoch 3/10
146/146 [==============================] - 0s 3ms/step - loss: 0.6836 - accuracy: 0.8273 - val_loss: 0.6093 - val_accuracy: 0.8602
Epoch 4/10
146/146 [==============================] - 0s 3ms/step - loss: 0.5790 - accuracy: 0.8576 - val_loss: 0.5236 - val_accuracy: 0.8827
Epoch 5/10
146/146 [==============================] - 0s 2ms/step - loss: 0.5104 - accuracy: 0.8755 - val_loss: 0.4638 - val_accuracy: 0.8969
Epoch 6/10
146/146 [==============================] - 0s 3ms/step - loss: 0.4611 - accuracy: 0.8873 - val_loss: 0.4204 - val_accuracy: 0.9030
Epoch 7/10
146/146 [==============================] - 0s 3ms/step - loss: 0.4242 - accuracy: 0.8907 - val_loss: 0.3870 - val_accuracy: 0.9084
Epoch 

,activation,loss,optimizer,Точность на обучающем,Точность на тестовом
5,<keras.layers.activation.leaky_relu.LeakyReLU ...,categorical_crossentropy,adam,0.923347,0.925646
3,relu,kullback_leibler_divergence,adam,0.922908,0.923642
7,<keras.layers.activation.leaky_relu.LeakyReLU ...,kullback_leibler_divergence,adam,0.921263,0.922752
1,relu,categorical_crossentropy,adam,0.921592,0.921416
2,relu,kullback_leibler_divergence,SGD,0.911284,0.910953


In [ ]:
model = Sequential()
model.add(Dense(units=280, input_dim = x_train.shape[1], activation = res.iloc[0]['activation']))
model.add(Dense(7, activation = "softmax")) 
model.compile(loss=res.iloc[0]['loss'], optimizer=res.iloc[0]['optimizer'],metrics = ["accuracy"])
model.fit(x_train, y_train, validation_split=0.2, epochs=10, batch_size=50, verbose=0)

model.save('/content/model')

print(f'Точность на тестовой выборке: {model.evaluate(x_test, y_test)[1]*100:.3f}%')
print('\nСтруктура модели:')
model.summary()


141/141 [==============================] - 0s 2ms/step - loss: 0.1978 - accuracy: 0.9288
Точность на тестовой выборке: 92.876%

Структура модели:
Model: "sequential_69"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_143 (Dense)           (None, 280)               4760      
                                                                 
 dense_144 (Dense)           (None, 7)                 1967      
                                                                 
Total params: 6,727
Trainable params: 6,727
Non-trainable params: 0
_________________________________________________________________
